Takes in .AWD, sleep log if it exists, calendar, and spits out some pretty pictures

In [92]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, '/data/MoodGroup/actigraphy/gavi/actigraPy') 

import actigraPy.actigraPy as act
import importlib
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

In [93]:
# data and log directories
sub = 'TEST' #subject NUMBER as STRING
sub_long = 'TEST'

out_dir = '/data/MoodGroup/actigraphy/gavi/actigraPy/data/output' #output directory
data_dir = '/data/MoodGroup/actigraphy/gavi/actigraPy/data'

if not os.path.exists(out_dir):
    os.mkdir(out_dir)


## Subject and directory info

In [94]:
fn = {}
#.AWD
awd_fn = os.path.join(data_dir, sub_long+'.AWD')
#sleeplog
fn['cal']= os.path.join(data_dir, sub_long + '_calendar_log.xls')
#calendar
fn['log'] = os.path.join(data_dir, sub_long+'_sleeplog.xls')
logs = []
Mtimes_fn = os.path.join(out_dir,sub_long+'_Mtimes_edit.csv')
orig_fn = os.path.join(out_dir,sub_long+'_Mtimes.csv')

if os.path.isfile(awd_fn):
    for name in fn.keys():
        if os.path.isfile(fn[name]):
            print("%s has %s file at %s"%(sub,name,fn[name]))
            logs.append(fn[name])
        else:
            print("no " + name)
            fn[name] = ''
else:
    print("DO NOT CONTINUE!! THERE IS NO AWD FILE")

TEST has cal file at /data/MoodGroup/actigraphy/gavi/actigraPy/data/TEST_calendar_log.xls
TEST has log file at /data/MoodGroup/actigraphy/gavi/actigraPy/data/TEST_sleeplog.xls


In [95]:
orig_fn

'/data/MoodGroup/actigraphy/gavi/actigraPy/data/output/TEST_Mtimes.csv'

### Read Mtimes file

Figure out where to clip awd file

In [96]:
#read AWD file
awd_dat = act.read_AWD(awd_fn)
idx={'start':[0],'end':[len(awd_dat['dt_list'])-1]}
#get the start and stops from each log if they exist
for log in logs:
    log_dat, kw_dat, comments = act.read_log(log,awd_dat)
    
    if 'watch_on' in kw_dat.keys():
        on_date = kw_dat['watch_on'].iloc[0]['OnDate']
        on_time = kw_dat['watch_on'].iloc[0]['OnTime']
        on = datetime(on_date.year,on_date.month,on_date.day,on_time.hour,on_time.minute)
        #check if that time is in dt_list:
        if awd_dat['dt_list'].count(on) > 0:
            on_idx=awd_dat['dt_list'].index(on)
            idx['start'].append(on_idx)

    if 'watch_off' in kw_dat.keys():
        off_date = kw_dat['watch_off'].iloc[0]['OffDate']
        off_time = kw_dat['watch_off'].iloc[0]['OffTime']
        off = datetime(off_date.year,off_date.month,off_date.day,off_time.hour,off_time.minute)
        if awd_dat['dt_list'].count(off) > 0:
            off_idx=awd_dat['dt_list'].index(off)
            idx['end'].append(off_idx)
 
idx

     OffDate   OffTime     OnDate    OnTime                Comment
0        NaT       NaN 2016-02-08  10:00:00                  start
1 2016-02-07  13:00:00 2016-02-07  14:30:00  Fake, for bug testing
2 2016-02-08  13:00:00 2016-02-08  14:30:00                     3T
3 2016-02-08  21:30:00 2016-02-09  07:00:00            Sleep study
4 2016-02-09  14:00:00 2016-02-09  16:00:00                     7T
5 2016-02-10  10:00:00 2016-02-10  12:00:00                    NaN
6 2016-02-10  21:30:00 2016-02-11  07:00:00            Sleep study
7 2016-02-11  09:00:00 2016-02-11  09:40:00               Infusion
8 2016-02-11  15:00:00 2016-02-11  17:00:00                    MEG
[(0, 74, '3T'), (494, 1064, 'Sleep study'), (1484, 1604, '7T'), (2684, 2804, ''), (3374, 3944, 'Sleep study'), (4064, 4104, 'Infusion'), (4424, 4544, 'MEG')]
     OffDate   OffTime     OnDate    OnTime Comment
0        NaT       NaN 2016-02-08  15:40:00   start
1 2016-02-09  00:42:00 2016-02-09  01:20:00  shower
2 2016-02-09  13

{'start': [0, 144], 'end': [4741]}

## <font color='red'>This part needs user input! Use your judgment to decide which starts and ends from above to go with</font>


In [97]:
start = max(idx['start'])
end = min(idx['end'])
print('start = %d, end = %d'%(start,end))

start = 144, end = 4741


In [98]:
#modify these according to output from above!!
#start =0
#end = 45283
lim_tup = [(awd_dat['DateTime'][start],awd_dat['DateTime'][end],"")]
clip_dat = act.clip_dat(lim,awd_dat)

Now just run these cells in order to extract comments and write Mtimes

sandbox start

sandbox end

In [151]:
importlib.reload(act)
log_dat,kw_dat=act.read_log(Mtimes_fn,awd_dat,oldvsn=False)

      OffDate   OffTime     OnDate    OnTime marker      Comment
0   08-Feb-16  01:16 PM  08-Feb-16  02:30 PM    cal           3T
1   08-Feb-16  01:16 PM  08-Feb-16  02:35 PM      z             
2   08-Feb-16  02:43 PM  08-Feb-16  03:31 PM      z             
3   08-Feb-16  03:49 PM  08-Feb-16  04:01 PM      z             
4   08-Feb-16  04:51 PM  08-Feb-16  05:03 PM      z             
5   08-Feb-16  06:35 PM  08-Feb-16  07:03 PM      z             
6   08-Feb-16  07:07 PM  08-Feb-16  07:45 PM      z             
7   08-Feb-16  09:30 PM  09-Feb-16  07:00 AM    cal  Sleep study
8   08-Feb-16  10:05 PM  08-Feb-16  10:23 PM      z             
9   08-Feb-16  10:29 PM  08-Feb-16  10:45 PM      z             
10  09-Feb-16  12:42 AM  09-Feb-16  01:20 AM    log       shower
11  09-Feb-16  12:42 AM  09-Feb-16  01:20 AM      g         TEST
12  09-Feb-16  12:47 AM  09-Feb-16  01:25 AM      z             
13  09-Feb-16  01:47 AM  09-Feb-16  08:35 AM      z             
14  09-Feb-16  09:13 AM  

In [152]:
mk_idx.keys()

dict_keys(['cal', 'log', 'm', 'z'])

In [127]:
tmp = [jj for ii in [(x[0],x[1]) for x in mk_idx['cal']] for jj in ii]
dat = { ii: awd_dat[ii] for ii in ['DateTime','activity','M'] }


In [130]:
mm_marks = [0] * len(dat)
for block in mk_idx['cal']:
    mm_marks[block[0]:block[1]]=[1]*(block[1]-block[0])
mm_marks

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [134]:
importlib.reload(act)
dat = act.write_dat(awd_dat,mk_idx,os.path.join(out_dir,sub_long))

## Make graphs

In [150]:
mk_idx

{'cal': [(0, 74, '3T'),
  (494, 1064, 'Sleep study'),
  (1484, 1604, '7T'),
  (2684, 2804, ''),
  (3374, 3944, 'Sleep study'),
  (4064, 4104, 'Infusion'),
  (4424, 4544, 'MEG')],
 'log': [(686, 724, 'shower'),
  (1469, 1639, '7T MRI'),
  (1469, 1639, 'TEST'),
  (2049, 2089, 'shower'),
  (3239, 3291, 'shower'),
  (4424, 4544, 'MEG')],
 'm': [(1462, 1470, ''), (1475, 1643, ''), (2056, 2093, ''), (3243, 3294, '')],
 'z': [(0, 79, ''),
  (87, 135, ''),
  (153, 165, ''),
  (215, 227, ''),
  (319, 347, ''),
  (351, 389, ''),
  (529, 547, ''),
  (553, 569, ''),
  (691, 729, ''),
  (751, 1159, ''),
  (1197, 1211, ''),
  (1293, 1313, ''),
  (1319, 1339, ''),
  (1473, 1641, ''),
  (1879, 1905, ''),
  (2055, 2091, ''),
  (2127, 2155, ''),
  (2161, 2551, ''),
  (2557, 2593, ''),
  (2599, 2651, ''),
  (2723, 2789, ''),
  (2991, 3007, ''),
  (3029, 3043, ''),
  (3051, 3093, ''),
  (3241, 3293, ''),
  (3301, 3321, ''),
  (3381, 3393, ''),
  (3401, 3413, ''),
  (3471, 3493, ''),
  (3607, 3935, ''),
  

In [147]:
wanted_plots=['g']
plots = {}
for name in wanted_plots:
    if name in mk_idx.keys():
        plots[name]=mk_idx[name]

In [23]:
wanted = ['g']
coms=[]
idxs=[]
for mm in wanted:
    print(mm)
    idxs= idxs +(mc_dict[mm]['idxs'])
    coms= coms + (mc_dict[mm]['comments'])
    if mm in del_com.keys():
        for i in range(0,len(del_com[mm])):
            idxs.pop(0)
            coms.pop(0)
zipped_pairs = zip(idxs,coms)
gc_coms = [x for _, x in sorted(zipped_pairs)]
gc_idx = np.sort(idxs) - start

gc=[np.asarray(gc_idx),gc_coms]

g


In [146]:
importlib.reload(act)

act.plot_awd(awd_dat,plots,max_act=500,show=False,comments=gc,fn_pref=os.path.join(out_dir,sub+'_M+edits'),plot_type='single',debug=True)


NameError: name 'plots' is not defined

Sandbox

In [136]:
idat = awd_dat['activity']

   # want to plot activity data by day
   # get number of days, then split data by days, the bar plota

   #idat = np.array([ int(ii) for ii in dat if ii is not('M') ] ) 
day_list = [ ii.split()[0] for ii in awd_dat['DateTime'] ]
days = list(np.unique(day_list))
days.sort(key=lambda x: datetime.strptime(x, '%d-%b-%y'))
   #print(days)
n_days = len(days)
   #n_days =  14
   #days = days[:n_days]
 
print(n_days)
     
wh = round(n_days/7)*8

4


In [141]:
days

['08-Feb-16', '09-Feb-16', '10-Feb-16', '11-Feb-16']

In [145]:
mk_idx['cal'][3][0]

2684